# Start Simulation

In [1]:
# env.clean(clean_all=True)

In [2]:
import random
import numpy as np
import os
import time
from PIL import Image

import torch
torch.__version__
device = torch.device("cuda")

In [3]:
import getpass
user = getpass.getuser()
print(user)

yizhou


In [4]:
usd_path = f"omniverse://localhost/Users/{user}/UVA/uva_book.usd"

In [5]:
from omni.isaac.kit import SimulationApp    
simulation_app = SimulationApp({"headless": True, "open_usd": usd_path,  "livesync_usd": usd_path}) 

Starting kit application with the fillowing args:  ['/home/yizhou/.local/share/isaac_sim-2022.1.1/exts/omni.isaac.kit/omni/isaac/kit/simulation_app.py', '/home/yizhou/.local/share/isaac_sim-2022.1.1/apps/omni.isaac.sim.python.kit', '--/app/tokens/exe-path=/home/yizhou/.local/share/isaac_sim-2022.1.1/kit', '--/persistent/app/viewport/displayOptions=3094', '--/rtx/materialDb/syncLoads=True', '--/rtx/hydra/materialSyncLoads=True--/omni.kit.plugin/syncUsdLoads=True', '--/app/renderer/resolution/width=1280', '--/app/renderer/resolution/height=720', '--/app/window/width=1440', '--/app/window/height=900', '--/renderer/multiGpu/enabled=True', '--ext-folder', '/home/yizhou/.local/share/isaac_sim-2022.1.1/exts', '--ext-folder', '/home/yizhou/.local/share/isaac_sim-2022.1.1/apps', '--/physics/cudaDevice=0', '--portable', '--no-window']
Passing the following args to the base kit application:  ['-f', '/home/yizhou/.local/share/jupyter/runtime/kernel-71d76129-73f8-4501-9a34-f104fda986a1.json']
[Warn

OmniAssetFileFormat


Warp initialized:
   Version: 0.2.2
   CUDA device: NVIDIA RTX A6000
   Kernel cache: /home/yizhou/.cache/warp/0.2.2
[1.420s] [ext: omni.kit.menu.create-1.0.2] startup
[1.421s] [ext: omni.mdl-0.1.0] startup
[1.436s] [ext: omni.kit.window.file_importer-1.0.4] startup
[1.437s] [ext: omni.kit.window.file_exporter-1.0.4] startup
[1.438s] [ext: omni.kit.material.library-1.3.10] startup
[1.440s] [ext: omni.kit.window.drop_support-1.0.0] startup
[1.440s] [ext: omni.kit.window.file-1.3.16] startup
[1.442s] [ext: omni.kit.context_menu-1.3.9] startup
[1.445s] [ext: omni.kit.window.property-1.6.3] startup
[1.446s] [ext: omni.kit.window.content_browser-2.4.28] startup
[1.454s] [ext: omni.kit.widget.stage-2.6.15] startup
[1.458s] [ext: omni.isaac.version-1.0.0] startup
[1.458s] [ext: omni.kit.property.usd-3.14.9] startup
[1.482s] [ext: omni.kit.viewport.legacy_gizmos-1.0.0] startup
[1.485s] [ext: omni.hydra.rtx-0.1.0] startup
[1.491s] [ext: omni.renderer-rtx-0.0.0] startup
[1.491s] [ext: omni.hydra

In [6]:
# set log level
import logging
import carb

logging.getLogger("omni.hydra").setLevel(logging.ERROR)
logging.getLogger("omni.isaac.urdf").setLevel(logging.ERROR)
logging.getLogger("omni.physx.plugin").setLevel(logging.ERROR)

logging.getLogger("omni.isaac.synthetic_utils").setLevel(logging.ERROR)
logging.getLogger("omni.isaac.synthetic_utils.syntheticdata").setLevel(logging.ERROR)
logging.getLogger("omni.hydra.scene_delegate.plugin").setLevel(logging.ERROR)


l = carb.logging.LEVEL_ERROR
carb.settings.get_settings().set("/log/level", l)
carb.settings.get_settings().set("/log/fileLogLevel", l)
carb.settings.get_settings().set("/log/outputStreamLevel", l)

# # This logged error is printed as it should
# carb.log_error("ERROR")
# # This warning is printed but should not
# carb.log_warn("WARNING")

# Config

In [7]:
from task.config import DATA_PATH, FEATURE_PATH
task_type = "Bookshelf"
side_choice = "Border"
base_asset_id = 0
load_nucleus = True

In [8]:
pause

NameError: name 'pause' is not defined

# Init Env

In [9]:
from uva_env import UvaEnv
env = UvaEnv()

In [10]:
from task.utils import add_scene_default
add_scene_default()

In [11]:
print(list(env.stage.TraverseAll()))

[Usd.Prim(</physicsScene>), Usd.Prim(</World>), Usd.Prim(</World/defaultLight>), Usd.Prim(</World/physicsScene>), Usd.Prim(</World/base>), Usd.Prim(</World/base/Sides>), Usd.Prim(</World/base/Shelves>), Usd.Prim(</World/base/Looks>), Usd.Prim(</World/base/Looks/Shelves>), Usd.Prim(</World/base/Looks/Shelves/Shader>), Usd.Prim(</World/base/Looks/Sides>), Usd.Prim(</World/base/Looks/Sides/Shader>), Usd.Prim(</World/groundPlane>), Usd.Prim(</World/groundPlane/CollisionMesh>), Usd.Prim(</World/groundPlane/CollisionPlane>), Usd.Prim(</World/layout>), Usd.Prim(</World/layout/structure>), Usd.Prim(</World/layout/structure/WallTop_2>), Usd.Prim(</World/layout/structure/WallBottom_3>), Usd.Prim(</World/layout/structure/WallInner_4>), Usd.Prim(</World/layout/structure/WallOuter_5>), Usd.Prim(</World/layout/structure/Baseboard_6>), Usd.Prim(</World/layout/structure/Baseboard_7>), Usd.Prim(</World/layout/structure/WallTop_8>), Usd.Prim(</World/layout/structure/WallBottom_9>), Usd.Prim(</World/layo

In [12]:
env.clean()
env.world.step(render=True)

# Scene

In [13]:
from task.scene import ArrangeScene
scene = ArrangeScene(task_type, side_choice, base_asset_id = 0, traj_id = 0, load_nucleus = load_nucleus)
env.scene = scene

In [14]:
# add base
scene.add_base_asset()

asset_prim: omniverse://localhost/Users/yizhou/Asset/B/Bookshelve/Clayton.usd


In [15]:
# add room
scene.add_room()

asset_prim: omniverse://localhost/Users/yizhou/Asset/S/0/layout.usd


In [16]:
env.world.step(render=True)

In [17]:
# randomize scene
from layout.randomizer import Randomizer
randomizer = Randomizer(load_nucleus = load_nucleus)

# Reward

In [18]:
from uv.reward import Rewarder

In [19]:
rewarder = Rewarder(env.world)
env.rewarder = rewarder

# Render

In [20]:
from render.helper import RenderHelper
render = RenderHelper(task_type, side_choice)

render.add_task_cameras(camera_type="main")
render.set_cameras()

xform_mat:  ( (-0.9999616399999998, 0, 0, 0), (0, 0.000019180000000118547, 0.9999808199999999, 0), (0, 0.9999808199999999, 0.000019180000000118547, 0), (0, 180, 100, 1) )


In [21]:
pause

NameError: name 'pause' is not defined

#  Learning

In [22]:
# # from learning.network.resnet import ResNetFeatureExtractor

# # feature extraction
# from learning.utils import extract_image_clip_feature_and_save, obtain_action_from_trainer
# from transformers import CLIPProcessor, CLIPModel

# feature_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
# feature_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

# # replay buffer
# import json
# from learning.replay_buffer import ReplayBuffer
# from learning.config import *

# buffer = ReplayBuffer(max_size=2000)

# # trainer
# from learning.network.sac import *

# policy = Policy()

# qf1 = QFunction()
# qf2 = QFunction()
# target_qf1 = QFunction()
# target_qf2 = QFunction()

# from learning.sac_trainer import SACTrainer

# trainer = SACTrainer(policy, qf1, qf2, target_qf1, target_qf2, 
#      use_automatic_entropy_tuning = False, 
#      policy_lr=1e-3, 
#      qf_lr=1e-3,
#      target_update_period = 5)

# Trajectory

In [23]:
use_network = False
debug = False

In [24]:
total_traj = 0
total_step = 0

---------------------

In [27]:
# traj config
for traj_id in range(0, 100):
    print("total_traj: ", total_traj)
    env.scene.traj_id = traj_id
    
    # base
    if total_traj % 5 == 0:
        asset_count = len(env.scene.base_asset_file_paths)
        base_asset_id = random.choice([_ for _ in range(asset_count)])
        env.scene.base_asset_id = base_asset_id    
        env.clean(clean_base = True, clean_all = False)
        scene.add_base_asset()
        
        # randomize scene
        randomizer.randomize_house(rand = True)
        
    image_folder = os.path.join(DATA_PATH, task_type, side_choice, str(traj_id))
    
    env.world.step(render = True)
    env.world.step(render = True)

    # get images
    env.world.render()
    images = render.get_images()
    render.save_rgb(images[0]['rgb'], image_folder, "0")
    
    ## extract feature
    
#     extract_image_feature_and_save(images[0]['rgb'][:,:,:3], 
#         feature_extractor, os.path.join(image_folder, str(0) + ".pt"))
    if use_network:
        extract_image_clip_feature_and_save(f"{image_folder}/{0}.png", feature_model, feature_processor, 
            f"{image_folder}/{0}.pt",)
    

    # trajectory
    for step in range(5):
        total_step += 1
        
        # sample an object
        env.add_scene_obj(mode = "random")
        
        # TODO: get action from sampling
        if not use_network or total_traj < 10 or np.random.rand() < 0.2:
            x, y = np.tanh(np.random.randn()), np.tanh(np.random.randn())
        else:
            image_feature_file = f"{image_folder}/{step}.pt"
            
            object_info = env.scene.objects[-1]
            object_type = object_info["type"]
            obj_name = object_info["name"][:-4]
            object_feature_file = os.path.join(FEATURE_PATH, object_type, obj_name + ".pt")
            x, y = obtain_action_from_trainer(image_feature_file, object_feature_file, trainer, 
                                              scaler=np.exp(- total_traj / 100))
            object_info["use_network"] = True
        
        # load the object into the scene
        env.put_last_object((x, y)) 
        env.world.step(render=True)
        
        # register the object to the world for physics update
        env.register_last_object()
        env.world.step(render=True)

        # get images
        env.world.render()
        images = render.get_images()
        render.save_rgb(images[0]['rgb'], image_folder, str(step + 1))

        ## calculate reward
        env.calculate_last_reward(simulation_step = 30)
        
        ## extract feature
#         extract_image_feature_and_save(images[0]['rgb'][:,:,:3], 
#             feature_extractor, os.path.join(image_folder, str(step + 1) + ".pt"))
        if use_network:
            extract_image_clip_feature_and_save(f"{image_folder}/{step + 1}.png", feature_model, feature_processor, 
                f"{image_folder}/{step + 1}.pt",)
    

        ## reset
        env.world.reset()
        env.world.step(render=True)
        
        ## trainer nework
        if use_network and total_step % UPDATE_TRAINER_STEPS == 0 and total_traj > 5:
            batch = buffer.sample_batch(batch_size = BATCH_SIZE)
            trainer.update(batch)
            
            if debug and total_step % 10 == 0:
                rewards = batch['rewards'].to(device)
                terminals = batch['terminals'].to(device)
                obs = batch['observations'].to(device)
                actions = batch['actions'].to(device)
                next_obs = batch['next_observations'].to(device)
                obj_features = batch['object_features'].to(device)
                
                dist = trainer.policy(obs, obj_features)
                pred = trainer.qf1(obs, obj_features, actions)
                print(f"debug {total_traj}/{total_step}", #"\n dist: ", dist.mean.flatten().tolist(), dist.stddev.flatten().tolist(),
                      "\n pred:", pred.flatten().tolist(),
                      "\n rewards: ", rewards.flatten().tolist())


    # Record
    record = env.scene.get_scene_data()
    env.scene.save_scene_data()
    # print("record: ", record)
    
    # Add record to buffer
    if use_network:
        buffer.add_scene_sample(record)

    # Reset (env clean)
    env.clean(clean_all = False)
    env.step(render = True)
    total_traj += 1

total_traj:  1
asset_prim: omniverse://localhost/Users/yizhou/Asset/I/Magazine/MagazineStack02.usd
asset_prim: omniverse://localhost/Users/yizhou/Asset/I/Magazine/OpenMagazine.usd
asset_prim: omniverse://localhost/Users/yizhou/Asset/I/Book/Book_Spelling.usd
asset_prim: omniverse://localhost/Users/yizhou/Asset/I/Plant/Plant_Succulent_02.usd
asset_prim: omniverse://localhost/Users/yizhou/Asset/I/Magazine/MagazineStack01.usd
total_traj:  2
asset_prim: omniverse://localhost/Users/yizhou/Asset/I/Vase/japan_vase.usd
asset_prim: omniverse://localhost/Users/yizhou/Asset/I/Sculpture/WoodenBird_Medium.usd
asset_prim: omniverse://localhost/Users/yizhou/Asset/I/Plant/Plant_Succulent_02.usd
asset_prim: omniverse://localhost/Users/yizhou/Asset/I/Vase/Earthenware03.usd
asset_prim: omniverse://localhost/Users/yizhou/Asset/I/Sculpture/GeoSphere_Small.usd
total_traj:  3
asset_prim: omniverse://localhost/Users/yizhou/Asset/I/Magazine/OpenMagazine.usd
asset_prim: omniverse://localhost/Users/yizhou/Asset/I

2023-01-03 01:55:43 [99,493ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xfortotal_traj:  5
asset_prim: omniverse://localhost/Users/yizhou/Asset/B/Bookshelve/Dogtown.usd
asset_prim: omniverse://localhost/Users/yizhou/Asset/I/Book/Book_77.usd
asset_prim: omniverse://localhost/Users/yizhou/Asset/I/Book/Book_Greener.usd
mOp:orient for /World/objects/Magazine_02

2023-01-03 01:55:43 [99,493ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Magazine_02

2023-01-03 01:55:43 [99,493ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Magazine_02

2023-01-03 01:55:43 [99,493ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Magazine_02

2023-01-03 01:55:43 [99,493ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Magazine_02

2023-01-03 01:55:43 [99,493ms] [Warnin

2023-01-03 01:55:57 [114,007ms] [Warning] [omni.hydra.stotal_traj:  7
asset_prim: omniverse://localhost/Users/yizhou/Asset/I/Magazine/MagazineStack02.usd
asset_prim: omniverse://localhost/Users/yizhou/Asset/I/Plant/Plant_02.usd
asset_prim: omniverse://localhost/Users/yizhou/Asset/I/Plant/Plant_Succulent_01.usd
asset_prim: omniverse://localhost/Users/yizhou/Asset/I/Book/Book_21.usd
asset_prim: omniverse://localhost/Users/yizhou/Asset/I/Vase/Earthenware03.usd
cene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Magazine

2023-01-03 01:55:58 [114,597ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-03 01:55:59 [115,290ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:rotateXYZ for /World/objects/Magazine

2023-01-03 01:55:59 [115,921ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-03 01:56:00 [116

dra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Book

2023-01-03 01:56:07 [123,835ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Book

2023-01-03 01:56:07 [123,835ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Book

2023-01-03 01:56:07 [123,835ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Book

2023-01-03 01:56:07 [123,835ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Book

2023-01-03 01:56:07 [123,835ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Book

2023-01-03 01:56:07 [123,835ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Book

2023-01-03 01:56:07 [123,835ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot 

2023-01-03 01:56:09 [125,asset_prim: omniverse://localhost/Users/yizhou/Asset/I/Magazine/Magazine01.usd
995ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Book

2023-01-03 01:56:09 [125,996ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Magazine_01

2023-01-03 01:56:09 [125,996ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Book

2023-01-03 01:56:09 [125,996ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Magazine_01

2023-01-03 01:56:09 [125,996ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Book

2023-01-03 01:56:09 [125,996ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Magazine_01

2023-01-03 01:56:09 [125,996ms] [Warning] [omni.hydra.scene_delegate.plu

...

2023-01-03 02:11:55 [1,072,192ms] [Warntotal_traj:  99
asset_prim: omniverse://localhost/Users/yizhou/Asset/I/Magazine/MagazineStack01.usd
asset_prim: omniverse://localhost/Users/yizhou/Asset/I/Book/PhotoAlbum_Closed.usd
asset_prim: omniverse://localhost/Users/yizhou/Asset/I/Plant/Plant_02.usd
asset_prim: omniverse://localhost/Users/yizhou/Asset/I/Book/Book_77.usd
asset_prim: omniverse://localhost/Users/yizhou/Asset/I/Plant/Plant_Succulent_01.usd
total_traj:  100
asset_prim: omniverse://localhost/Users/yizhou/Asset/B/Bookshelve/Fenton.usd
asset_prim: omniverse://localhost/Users/yizhou/Asset/I/Vase/CellVase03.usd
ing] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Book

2023-01-03 02:11:55 [1,072,192ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Plant

2023-01-03 02:11:55 [1,072,192ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Scul

2023-01-03 02:12:03 [1,080,225ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:rotateZYX for /World/objects/Book

2023-01-03 02:12:04 [1,080,393ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-03 02:12:04 [1,081,061ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Book

2023-01-03 02:12:04 [1,081,061ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Vase

2023-01-03 02:12:04 [1,081,061ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Book

2023-01-03 02:12:04 [1,081,061ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Vase

2023-01-03 02:12:04 [1,081,061ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Book

2023-01-03 02:

orm op xformOp:orient for /World/objects/Book

2023-01-03 02:12:04 [1,081,061ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Vase

2023-01-03 02:12:04 [1,081,061ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Book

2023-01-03 02:12:04 [1,081,061ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Vase

2023-01-03 02:12:05 [1,081,241ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:rotateXYZ for /World/objects/Sculpture

2023-01-03 02:12:05 [1,081,418ms] [Warning] [omni.isaac.synthetic_utils.syntheticdata] Set verify_sensor_init to false if running with asyncio
2023-01-03 02:12:05 [1,082,084ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Book

2023-01-03 02:12:05 [1,082,084ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xf

12:05 [1,082,084ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Vase

2023-01-03 02:12:05 [1,082,084ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Book

2023-01-03 02:12:05 [1,082,084ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Vase

2023-01-03 02:12:05 [1,082,084ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Book

2023-01-03 02:12:05 [1,082,084ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Vase

2023-01-03 02:12:05 [1,082,084ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Book

2023-01-03 02:12:05 [1,082,084ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Vase

2023-01-03 02:12:06 [1,082,276ms] 

n] cannot find xform op xformOp:orient for /World/objects/Vase

2023-01-03 02:12:06 [1,083,210ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Book

2023-01-03 02:12:06 [1,083,210ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Vase

2023-01-03 02:12:06 [1,083,210ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Book

2023-01-03 02:12:06 [1,083,210ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Vase

2023-01-03 02:12:06 [1,083,210ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Book

2023-01-03 02:12:06 [1,083,210ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform op xformOp:orient for /World/objects/Vase

2023-01-03 02:12:06 [1,083,210ms] [Warning] [omni.hydra.scene_delegate.plugin] cannot find xform

# Debug

In [ ]:
env.clean(clean_base = True, clean_all = False)

In [ ]:
# get images
env.world.render()
env.world.render()
images = render.get_images()

Image.fromarray(images[0]['rgb'], "RGBA").resize((300, 300))

##  Debug Training

In [ ]:
import json
from learning.replay_buffer import ReplayBuffer

buffer = ReplayBuffer(max_size=1000)

for i in range(100):
    replay = json.load(open(f"{DATA_PATH}/{task_type}/{side_choice}/{i}/scene.json"))
    buffer.add_scene_sample(replay)

In [ ]:
len(buffer.dataset)

In [ ]:
# trainer
from learning.network.sac import *

policy = Policy()

qf1 = QFunction()
qf2 = QFunction()

target_qf1 = QFunction()
target_qf1.eval()

target_qf2 = QFunction()
target_qf2.eval()

from learning.sac_trainer import SACTrainer

trainer = SACTrainer(policy, qf1, qf2, target_qf1, target_qf2, 
     use_automatic_entropy_tuning = True, 
     policy_lr=1e-3, 
     qf_lr=1e-3,
     target_update_period = 5)

In [ ]:
self = trainer

In [ ]:
for _ in range(500):
    batch = buffer.sample_batch(batch_size=6)

    rewards = batch['rewards'].to(device)
    terminals = batch['terminals'].to(device)
    obs = batch['observations'].to(device)
    actions = batch['actions'].to(device)
    next_obs = batch['next_observations'].to(device)

    obj_features = batch['object_features'].to(device)

    # print("rewards", rewards)

    self.update(batch)
    
    if _ % 10 == 0:
        dist = self.policy(obs, obj_features)
        pred = self.qf1(obs, obj_features, actions)
        print(_, "\n dist   : ", [round(x, 3) for x in dist.mean.flatten().tolist()], 
                  [round(x, 3) for x in dist.stddev.flatten().tolist()],
              "\n pred   :", [round(x, 3) for x in pred.flatten().tolist()],
              "\n rewards: ", [round(x, 3) for x in rewards.flatten().tolist()])
    
#     # debug
#     pred = self.qf1(obs, obj_features, actions)
#     loss = self.qf_criterion(pred, rewards)
    
#     self.qf1_optimizer.zero_grad()
#     loss.backward()
#     self.qf1_optimizer.step()
    
#     if _ % 20 == 0:
#         print(_, "\n debug loss: ", loss.item(), "\n pred:", 
#               pred.flatten().tolist(), "\n rewards: ", rewards.flatten().tolist())

In [ ]:
dist = self.policy(obs.to(device), obj_features.to(device))

In [ ]:
dist.normal_mean, dist.stddev

In [ ]:
dist.sample(scaler = 0.2)

In [ ]:
self = dist

In [ ]:
scaler = 0.1

In [ ]:
self.normal_mean + scaler * self.normal_std * MultivariateDiagonalNormal(
    torch.zeros(self.normal_mean.size()).to(self.device),
    torch.ones(self.normal_std.size()).to(self.device)
).sample()

# distribution

In [ ]:
from learning.distributions import MultivariateDiagonalNormal

In [ ]:
dist = JustNormal(dist.mean, dist.stddev)

In [ ]:
dist.rsample_and_logprob()